In [1]:
import os
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import torch

In [2]:
from totalface.model_zoo.model_common import load_tensorRT,load_openvino

In [3]:
def normalization(rgb_img,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    MEAN = 255 * np.array(mean_list)
    STD = 255 * np.array(std_list)
    rgb_img = rgb_img.transpose(-1, 0, 1)
    norm_img = (rgb_img - MEAN[:, None, None]) / STD[:, None, None]
    
    return norm_img

def image_preprocess(img_path,resize=224,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    img = cv2.cvtColor(cv2.imread(img_path),cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (resize, resize))
    
    img = normalization(img,mean_list,std_list)
    
    img = torch.from_numpy(img).unsqueeze(0).float()
    return img

In [6]:
def softmax(x):
    f_x = np.exp(x) / np.sum(np.exp(x))
    return f_x

In [22]:
test_path = "rmfd_mfr2/mfr2_2_scrfd_aligned/JoeBiden/JoeBiden_0001_mask.png"
test_path = "test_sample.jpg"
test_path = "/test_sample.png"
#test_path = "test_sample1.png"

In [23]:
img = image_preprocess(test_path,resize=112)

In [24]:
# tensorRT

In [25]:
model_path = "FaceMask_mbf2.v7.trt"
model = load_tensorRT.TrtModel(model_path,torch_image=True,not_norm=True)

In [26]:
output = model(img)[0]
output = softmax(output)[0]

In [27]:
output

array([0.7238758 , 0.27612415], dtype=float32)

In [15]:
num_iter = 10000
from tqdm import tqdm
import time

In [17]:
all_time=0
for i in tqdm(range(num_iter)):
    with torch.no_grad():
        s = time.time()
        output = model(img)[0]
        e = time.time()

    all_time+=((e-s)*1000)

speed = (all_time/num_iter)
print("Model: trt")
print("Model path: {}".format(model_path))
print("Time(batch): {} ms\n".format(speed))

100%|██████████| 10000/10000 [00:06<00:00, 1603.64it/s]

Model: trt
Model path: /data/shared/Face/Models/classification/mbf2.v7.trt
Time(batch): 0.6102895498275757 ms



In [11]:
import tensorrt
tensorrt.__version__

'7.2.1.6'

In [25]:
# openvino

In [7]:
model_path = ["FaceMask_mbf2_quantINT8.xml",
             "FaceMask_mbf2_quantINT8.bin"]
model = load_openvino.Openvino(model_path,not_norm=True,torch_image=True)

In [8]:
output = model(img)[0]
output = softmax(output)[0]

In [9]:
output

array([0.7245528, 0.2754472], dtype=float32)

In [42]:
img

tensor([[[[ 0.7591,  0.9303,  0.6563,  ...,  0.9988,  1.1015,  1.1015],
          [ 0.9303,  0.5536,  0.4337,  ...,  1.0331,  1.1187,  1.1187],
          [ 0.2111,  0.1597,  0.5193,  ...,  1.0844,  1.0844,  1.1187],
          ...,
          [-0.0116,  0.1768,  0.1597,  ...,  0.4508,  0.5536,  0.8618],
          [-0.2684, -0.3369, -0.1314,  ...,  0.1083,  0.1768,  0.2111],
          [-0.3883, -0.5424, -0.5253,  ...,  0.1083,  0.2282,  0.1768]],

         [[ 1.3431,  1.4482,  1.1331,  ...,  0.9755,  1.0980,  1.0980],
          [ 1.5182,  1.0630,  0.8704,  ...,  1.0280,  1.1155,  1.1155],
          [ 0.8004,  0.6429,  0.9580,  ...,  1.0805,  1.0805,  1.1155],
          ...,
          [ 0.1877,  0.4328,  0.4153,  ...,  0.6604,  0.7654,  1.0280],
          [-0.0399, -0.0924,  0.1176,  ...,  0.2927,  0.3102,  0.3102],
          [-0.1625, -0.3550, -0.3550,  ...,  0.2402,  0.3452,  0.2577]],

         [[ 1.6988,  1.8383,  1.4374,  ...,  1.0714,  1.1934,  1.1934],
          [ 1.8731,  1.4025,  

In [25]:
all_time=0
for i in tqdm(range(num_iter)):
    with torch.no_grad():
        s = time.time()
        output = model(img)[0]
        e = time.time()

    all_time+=((e-s)*1000)

speed = (all_time/num_iter)
print("Model: vino")
print("Model path: {}".format(model_path))
print("Time(batch): {} ms\n".format(speed))

100%|██████████| 10000/10000 [00:22<00:00, 450.17it/s]

Model: vino
Model path: ['/data/shared/Face/Models/classification/mbf2_quantINT8.xml', '/data/shared/Face/Models/classification/mbf2_quantINT8.bin']
Time(batch): 2.1988298177719114 ms



In [10]:
from openvino.inference_engine import get_version
get_version()

'2022.2.0-7713-af16ea1d79a-releases/2022/2'

In [31]:
import os
import numpy as np
import cv2
import torch
from torchvision import transforms

In [27]:
img_path = "rmfd_mfr2/mfr2_2_scrfd_aligned/JoeBiden/JoeBiden_0001_mask.png"

In [29]:
def normalization(rgb_img,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    MEAN = 255 * np.array(mean_list)
    STD = 255 * np.array(std_list)
    rgb_img = rgb_img.transpose(-1, 0, 1)
    norm_img = (rgb_img - MEAN[:, None, None]) / STD[:, None, None]
    
    return norm_img

def image_preprocess(img_path,resize=224,mean_list=[0.485, 0.456, 0.406],std_list=[0.229, 0.224, 0.225]):
    img = cv2.cvtColor(cv2.imread(img_path),cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (resize, resize))
    
    img = normalization(img,mean_list,std_list)
    
    img = torch.from_numpy(img).unsqueeze(0).float()
    return img

In [34]:
def image_transform(img_path,input_size):
    img = cv2.cvtColor(cv2.imread(img_path),cv2.COLOR_BGR2RGB)
    transform = transforms.Compose([
                            transforms.ToPILImage(),
                            transforms.Resize(input_size),
#                             transforms.RandomHorizontalFlip(),
#                             transforms.RandomRotation(10),
#                             transforms.RandomAffine(degrees=0,translate=None,scale=None,shear=20),
                            transforms.ToTensor(),
                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),

                        ])
    return transform(img)

In [30]:
img1 = image_preprocess(img_path,resize=112)

In [36]:
img2 = image_transform(img_path,112)

In [37]:
img1

tensor([[[[ 0.0741,  0.0741,  0.0912,  ...,  0.9303,  0.9646,  0.9988],
          [ 0.0741,  0.0741,  0.0741,  ...,  0.8961,  0.8789,  0.9817],
          [ 0.0569,  0.0569,  0.0569,  ...,  0.9988,  0.8961,  0.8789],
          ...,
          [ 0.5364,  0.5193,  0.4508,  ...,  0.0569,  0.0912, -0.0629],
          [ 0.5364,  0.5193,  0.4337,  ...,  0.0056,  0.0569, -0.3027],
          [ 0.5364,  0.5193,  0.4337,  ..., -0.0629, -0.1143, -0.4568]],

         [[-0.8277, -0.8277, -0.8102,  ...,  0.1877,  0.2752,  0.3627],
          [-0.8277, -0.8277, -0.8277,  ...,  0.1352,  0.1702,  0.3277],
          [-0.8452, -0.8452, -0.8452,  ...,  0.2577,  0.2052,  0.2577],
          ...,
          [-0.3725, -0.3901, -0.4426,  ..., -0.0399, -0.1625, -0.5476],
          [-0.3725, -0.3901, -0.4426,  ..., -0.1975, -0.2675, -0.8627],
          [-0.3725, -0.3901, -0.4426,  ..., -0.3375, -0.5651, -1.1254]],

         [[-1.3687, -1.3687, -1.3513,  ..., -0.0092,  0.0953,  0.2173],
          [-1.3687, -1.3687, -

In [38]:
img2

tensor([[[ 0.0741,  0.0741,  0.0912,  ...,  0.9303,  0.9646,  0.9988],
         [ 0.0741,  0.0741,  0.0741,  ...,  0.8961,  0.8789,  0.9817],
         [ 0.0569,  0.0569,  0.0569,  ...,  0.9988,  0.8961,  0.8789],
         ...,
         [ 0.5364,  0.5193,  0.4508,  ...,  0.0569,  0.0912, -0.0629],
         [ 0.5364,  0.5193,  0.4337,  ...,  0.0056,  0.0569, -0.3027],
         [ 0.5364,  0.5193,  0.4337,  ..., -0.0629, -0.1143, -0.4568]],

        [[-0.8277, -0.8277, -0.8102,  ...,  0.1877,  0.2752,  0.3627],
         [-0.8277, -0.8277, -0.8277,  ...,  0.1352,  0.1702,  0.3277],
         [-0.8452, -0.8452, -0.8452,  ...,  0.2577,  0.2052,  0.2577],
         ...,
         [-0.3725, -0.3901, -0.4426,  ..., -0.0399, -0.1625, -0.5476],
         [-0.3725, -0.3901, -0.4426,  ..., -0.1975, -0.2675, -0.8627],
         [-0.3725, -0.3901, -0.4426,  ..., -0.3375, -0.5651, -1.1253]],

        [[-1.3687, -1.3687, -1.3513,  ..., -0.0092,  0.0953,  0.2173],
         [-1.3687, -1.3687, -1.3687,  ..., -0

In [39]:
# dataset count

In [40]:
import pandas as pd

In [64]:
path_format = "FaceMask/filtering/saved_{}.csv"

In [65]:
tvt = ['train','val','test']

In [66]:
ct=[]
ct_dict={'train':[0,0],'val':[0,0],'test':[0,0]}
for t in tvt:
    df = pd.read_csv(path_format.format(t))
    ct.append(len(df))
    
    for val in df.values:
        mask_flag = val[-4]
        if mask_flag:
            ct_dict[t][1]+=1
        else:
            ct_dict[t][0]+=1
print("train: {} / val: {} / test: {}".format(ct[0],ct[1],ct[2]))
print("train - unmask: {} / mask: {} || val - unmask: {} / mask: {} || test - unmask: {} / mask: {}".format(ct_dict['train'][0],ct_dict['train'][1], \
                                                                                                         ct_dict['val'][0],ct_dict['val'][1], \
                                                                                                         ct_dict['test'][0],ct_dict['test'][1]))

train: 120031 / val: 14832 / test: 14871
train - unmask: 60031 / mask: 60000 || val - unmask: 7332 / mask: 7500 || test - unmask: 7371 / mask: 7500
